<a href="https://colab.research.google.com/github/sneakatyou/ViT-Tensorflow-2.0/blob/main/VIT_PRELIM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import math

import six
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras import datasets
from datetime import datetime
import logging
import numpy as np
from fastprogress import master_bar, progress_bar
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from google.colab.patches import cv2_imshow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('INFO')
from PIL import Image
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import CSVLogger

In [ ]:
%load_ext tensorboard
!rm -rf ./logs/
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#Data Loading

In [ ]:
!scp "/content/drive/My Drive/INS/malimg_dataset_1.zip" ./
!unzip -qq malimg_dataset_1.zip

In [ ]:
%load_ext tensorboard
import tensorboard
!rm -rf ./logs/
from datetime import datetime
import tensorflow as tf
import os
import dlib
import cv2
import numpy as np
import random

WORKERS = tf.data.experimental.AUTOTUNE
VAL_SPLIT = 0.2
IMAGE_SIZE = (224, 224, 3)
OCCLUSION_SIZE = (110, 110)
BATCH_SIZE = 32
POOL_SIZE = 6
PATCH_SIZE = 24
BUFFER_SIZE = 10000

base_dir = "datasets"
RAF_DB = os.path.join(base_dir, 'RAF-DB/Images')
AFFECT_PATH_train = os.path.join(base_dir, 'affect_net_data')
AFFECT_PATH_val = os.path.join(base_dir, 'affect_net_validation')
OCCLUSION_PATH = os.path.join(base_dir, 'occlusion')
FEDRO_PATH = os.path.join(base_dir, 'FED-RO/FED-RO_crop')
LANDMARK_PREDICTOR_PATH = 'util_files/shape_predictor_68_face_landmarks.dat'

class Dataset():
  def __init__(self,X_train,X_test,Y_train,Y_test,new_virus = False, rafdb = RAF_DB, affectnet = AFFECT_PATH_train, affectnetval = AFFECT_PATH_val, fedro=FEDRO_PATH,
  batch_size = BATCH_SIZE, image_shape = IMAGE_SIZE, patch_size = PATCH_SIZE, 
  occlusion_size = OCCLUSION_SIZE, create_occlusion = False,flag_for_fedro=False,flag_for_rafdb=False,flag_for_affectnet=False):
    self.path1 = "/content/malimg_dataset"
    self.path2 = affectnet
    self.path3 = affectnetval
    self.path4 = fedro
    self.val = False
    self.new_virus = new_virus
    self.batch_size = batch_size
    self.image_height = image_shape[0]
    self.image_width = image_shape[1]
    self.label_dict = {'adialer.c': 0, 'agent.fyi': 1, 'allaple.l': 2, 'alueron.gen!j': 3, 'autorun.k': 4, 'c2lop.p': 5, 'dialplatform.b': 6,'dontovo.a':7,'fakerean':8,'instantaccess':9,'lolyda.aa1':10,'lolyda.aa2':11,'lolyda.aa3':12, 'lolyda.at':13,'malex.gen!j':14,'obfuscator.ad':15,'rbot!gen':16,'skintrim.n':17,'swizzor.gen!e':18,'swizzor.gen!i':19,'vb.at':20,'wintrim.bx':21,'yuner.a':22,'allaple.a':23,'c2lop.gen!g': 24}
    #self.label_dict = {'Lolyda.BF': 0,"PWSWin32VB": 1,'SoftwareBundlerWin32Prepscram':2,'TrojanWin32Eksor.A':3,'Wintrim.BF':4}
    self.num_classes = len(list(self.label_dict))
    self.create_occlusion = create_occlusion
    self.X_train, self.X_test, self.Y_train, self.Y_test = X_train,X_test,Y_train,Y_test
    self.RandomFlip = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical")
    self.RandomRotation = tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
    #self.RandomCrop = tf.keras.layers.experimental.preprocessing.RandomCrop(160,160,seed = 777)
    #self.RandomZoom = tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
    self.Resize =  tf.keras.layers.experimental.preprocessing.Resizing(224,224)

  
  def aug(self,image,label):
      
    image = self.RandomFlip(image)
    image = self.RandomRotation(image)
    #image = self.RandomCrop(image)
    #image = self.RandomZoom(image)
    image = self.Resize(image)
    return image,label
  
  
  def parse_function(self, image, label):
    #image = tf.io.read_file(image_path)
    #image = tf.image.decode_jpeg(image, channels = 3)
    # image = tf.image.resize_with_crop_or_pad(image, self.image_height + 10, self.image_width + 10)
    # image = tf.image.random_crop(image, size = [self.image_height, self.image_width, 3])
    # image = tf.image.random_brightness(image, max_delta = 0.5)
    if self.new_virus:
        image = image[:,:,0:3]

    else:
        if not self.val:
            image = tf.expand_dims(image,axis = 2)
            image = tf.concat([image,image,image],axis = 2)
        
        else:
            image = image[:,:,0:3]
    image = tf.image.resize(image, [self.image_width, self.image_height])

    #label = tf.one_hot(label, self.num_classes)
    return image,label
  
  def get_train_ds(self):
    BUFFER_SIZE = len(self.X_train)
    ds = tf.data.Dataset.from_tensor_slices((self.X_train , self.Y_train))
    ds = ds.shuffle(BUFFER_SIZE)
    ds = ds.repeat()
    ds = ds.map(self.parse_function, num_parallel_calls = WORKERS)
    ds = ds.batch(self.batch_size, drop_remainder = True) 
    #ds = ds.map(self.aug,num_parallel_calls = WORKERS)
    ds = ds.prefetch(buffer_size=WORKERS)
    return ds
  
  def get_test_ds(self):
    BUFFER_SIZE = len(self.X_test)
    self.val = False
    ds = tf.data.Dataset.from_tensor_slices((self.X_test , self.Y_test))
    ds = ds.shuffle(BUFFER_SIZE)
    ds = ds.repeat(count=1)
    ds = ds.map(self.parse_function, num_parallel_calls = WORKERS)
    ds = ds.batch( self.batch_size, drop_remainder = True )
    ds = ds.prefetch(buffer_size=WORKERS)
    return ds

In [ ]:
from sklearn.model_selection import train_test_split
def get_ds():
    bs = configs.batch_size
    #X_list and Y_list are lists containing images ans labels respectively
    X_train,X_test,Y_train,Y_test = train_test_split(X_list,Y_list,train_size = 0.7) 
    dataset = Dataset(X_train,X_test,Y_train,Y_test,batch_size=bs,new_virus=False)
    train_ds = dataset.get_train_ds()
    test_ds = dataset.get_test_ds()
    return train_ds,test_ds

#TPU Initialization

In [ ]:
tf.keras.backend.clear_session()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.experimental.TPUStrategy(resolver)

#Positional encoding

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates
  
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
  pos_encoding = angle_rads[np.newaxis, ...]
  return tf.cast(pos_encoding, dtype=tf.float32)


In [ ]:
n, d = 2048, 512
pos_encoding = positional_encoding(n, d)
print(pos_encoding.shape)
pos_encoding = pos_encoding[0]
pos_encoding = tf.reshape(pos_encoding, (n, d//2, 2))
pos_encoding = tf.transpose(pos_encoding, (2,1,0))
pos_encoding = tf.reshape(pos_encoding, (d, n))
plt.pcolormesh(pos_encoding, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()


#MHA

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,num_heads,d_model):
        super(MultiHeadAttention,self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model%num_heads==0 
        self.multihead_d_model = tf.cast(d_model/num_heads,tf.int32)
        self.dense1 = tf.keras.layers.Dense(d_model)
        self.dense2 = tf.keras.layers.Dense(d_model)
        self.dense3 = tf.keras.layers.Dense(d_model)
        self.dense4 = tf.keras.layers.Dense(d_model)
    
    def SelfAttention(self,q,k,v,d_model):
        x = tf.matmul(q,k,transpose_b = True)
        x = tf.divide(x,tf.math.sqrt(tf.cast(d_model,tf.float32))) #scale q*k(T)
        attention_weights = tf.keras.activations.softmax(x,axis = -1) #calculating attention weights

        x = tf.matmul(attention_weights,v)
        return x,attention_weights
    
    def create_multi_heads(self,x,batch_size):
        x = tf.reshape(x,(batch_size,-1,self.num_heads,self.multihead_d_model))
        x = tf.transpose(x,perm = [0,2,1,3])
        return x
    
    def call(self,x):
        batch_size = x.get_shape()[0]
        q = self.dense1(x)
        k = self.dense2(x)
        v = self.dense3(x)

        q = self.create_multi_heads(q,batch_size)
        k = self.create_multi_heads(k,batch_size)
        v = self.create_multi_heads(v,batch_size)


        x,attention_weights = self.SelfAttention(q,k,v,self.d_model)
        x = tf.transpose(x, perm = [0,2,1,3])
        x = tf.reshape(x,(batch_size,-1,self.d_model))
        x = self.dense4(x)
        return x,attention_weights


In [ ]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y)
out.shape, attn.shape

#OTHERS

In [ ]:
class Add(tf.keras.layers.Layer):
    def __init__(self):
        super(Add,self).__init__()

    def call(self,x,x1):
        return tf.add(x,x1)


In [ ]:
class MLP(tf.keras.layers.Layer):
    def __init__(self,hidden_shape1,hidden_shape2,dropout_rate = 0.1):
        super(MLP,self).__init__()
        self.dense1 = tf.keras.layers.Dense(hidden_shape1,bias_initializer=tf.keras.initializers.RandomNormal(stddev=1e-6))
        self.dense2 = tf.keras.layers.Dense(hidden_shape2,bias_initializer=tf.keras.initializers.RandomNormal(stddev=1e-6))
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self,x,training =True):
        x = self.dense1(x)
        x = tf.keras.activations.gelu(x)
        x = self.dropout1(x,training = training)
        x = self.dense2(x)
        x = self.dropout2(x,training = training)
        return x



In [ ]:
class ImgPatches(tf.keras.layers.Layer):
    def __init__(self,d_model,patch_size,):
        super(ImgPatches,self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(d_model,patch_size,patch_size,padding = 'valid')
        self.patch_size = patch_size
        self.d_model = d_model
    def call(self,x):
        batch_size = x.get_shape()[0]
        print(batch_size)
        x = self.conv1(x)
        print(x.get_shape()[0],x.get_shape()[1],x.get_shape()[2],x.get_shape()[3])
        x = tf.reshape(x,(x.get_shape()[0],tf.multiply(x.get_shape()[1],x.get_shape()[2]),x.get_shape()[3]))
        return x

In [ ]:
x = tf.random.uniform((1,224,224,3))
img = ImgPatches(d_model = 5,patch_size=32)
y = img(x)
y

In [ ]:
class AddCls(tf.keras.layers.Layer):
    def __init__(self,d_model):
        super(AddCls,self).__init__()
        self.d_model = d_model
        self.cls = tf.Variable(tf.zeros((1,1,d_model),dtype=tf.float32),trainable=True)
    def build(self,input_shape):
        batch_size = input_shape[0]
        self.cls = tf.Variable(tf.zeros((1,1,self.d_model),dtype=tf.float32),trainable=True)
        self.cls  = tf.tile(self.cls,[batch_size,1,1])
    def call(self,x):
        batch_size = x.get_shape()[0]
        print(self.cls.shape)
        #self.cls  = tf.tile(self.cls,[batch_size,1,1])
        print(self.cls.shape,x.shape)
        x = tf.concat([x,self.cls],axis=1)
        return x

In [ ]:
x = tf.random.uniform((32,49,512))
img = AddCls(512)
y = img(x)
y.shape

#ENCODER

In [ ]:
class EncoderBlock(tf.keras.layers.Layer):
    def __init__(self,num_heads,d_model,hidden_layer_shape,dropout_rate = 0.1):
        super(EncoderBlock,self).__init__()
        self.norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.mlp = MLP(hidden_shape1=hidden_layer_shape,hidden_shape2=d_model,dropout_rate=dropout_rate)
        #self.Add = Add()
        self.mha = MultiHeadAttention(num_heads=num_heads,d_model = d_model)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self,inp,training = True):
        x = self.norm1(inp)
        x,_ = self.mha(x,training=training)
        x = self.dropout1(x,training = training)
        x = x + inp

        y = self.norm2(x)
        y = self.mlp(y)
        y = x+y

        return y

In [ ]:
temp_mha = EncoderBlock(d_model=512, num_heads=8,hidden_layer_shape=2048)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out = temp_mha(y,training=True)
out.shape

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self,num_heads,d_model,hidden_layer_shape,num_layers,dropout_rate = 0.1):
        super(Encoder,self).__init__()
        self.encoder_layers = [EncoderBlock(num_heads=num_heads,d_model = d_model,hidden_layer_shape=hidden_layer_shape,dropout_rate=dropout_rate) for _ in range(num_layers)]
        self.num_layers = num_layers
        self.norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    def call(self,x,training = True):
        for layer in range(self.num_layers):
            x = self.encoder_layers[layer](x,training=training)
        x = self.norm1(x)
        return x

In [ ]:
temp_mha = Encoder(d_model=512, num_heads=8,hidden_layer_shape=2048,num_layers = 8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out = temp_mha(y,training=True)
out.shape

#VIT Model

In [ ]:
class ViT(tf.keras.Model):
    def __init__(self,num_heads,d_model,hidden_layer_shape,num_layers,patch_size,num_classes,dropout_rate = 0.1,training = True):
        super(ViT,self).__init__()
        self.encoder = Encoder(num_heads=num_heads,d_model=d_model,hidden_layer_shape=hidden_layer_shape,num_layers=num_layers,dropout_rate=dropout_rate)
        self.img_patches = ImgPatches(d_model=d_model,patch_size=patch_size)
        self.addcls = AddCls(d_model=d_model)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dense1 = tf.keras.layers.Dense(units = num_classes,kernel_initializer = tf.keras.initializers.zeros)
    
    def call(self,input,training = True):
        x = self.img_patches(input)
        x = self.addcls(x)
        emb = positional_encoding(x.get_shape()[1],x.get_shape()[2])
        x = tf.add(x,emb)
        x = self.dropout1(x,training = training)
        x = self.encoder(x,training = training)
        print(x.shape)
        x = x[:,0]
        x = self.dense1(x)
        return x


In [ ]:
temp_mha = ViT(num_heads = 8,d_model = 512,hidden_layer_shape = 2048,num_layers = 12,patch_size = 32,num_classes = 10,dropout_rate = 0.1,training = True)
y = tf.random.uniform((32, 224,224, 3))  # (batch_size, encoder_sequence, d_model)
out = temp_mha(y,training=True)
out.shape

In [ ]:
temp_mha = ViT(num_heads = 8,d_model = 512,hidden_layer_shape = 2048,num_layers = 12,patch_size = 32,num_classes = 10,dropout_rate = 0.1,training = True)
temp_mha.build((32,224,224,3))

#Config

In [ ]:
class Configs():
    def __init__(self,num_classes, num_heads,num_layers,d_model,hidden_layer_shape1,hidden_layer_shape2,dropout_rate,training,batch_size,patch_size,learning_rate):
        self.num_classes = num_classes
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.d_model = d_model
        self.hidden_layer_shape1 = hidden_layer_shape1
        self.hidden_layer_shape2 = hidden_layer_shape2
        self.dropout_rate = dropout_rate
        self.training = training
        self.batch_size = batch_size
        self.patch_size = patch_size
        self.learning_rate = learning_rate

In [ ]:
configs = Configs(num_classes = 25, 
                  num_heads = 12,
                  num_layers = 12,
                  d_model = 768,
                  hidden_layer_shape1 = 3072,
                  hidden_layer_shape2 = 1024,
                  dropout_rate = 0.1,
                  training = True,
                  batch_size = 32,
                  patch_size = 16,
                  learning_rate = 0.001)

#Training

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


TPU Training

In [ ]:
with strategy.scope():
    model = ViT(num_heads = configs.num_heads,
                d_model = configs.d_model ,
                hidden_layer_shape = configs.hidden_layer_shape1,
                num_layers = configs.num_layers,
                patch_size = configs.patch_size,
                num_classes = configs.num_classes,
                dropout_rate = configs.dropout_rate,
                training = configs.training)
    model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer = tf.keras.optimizers.Adam(learning_rate=configs.learning_rate),
                  metrics = ["accuracy"])
    model.build((64,224,224,3))

In [ ]:
model.fit(train_ds,epochs = 100,steps_per_epoch=100,validation_data = test_ds)

GPU TRAINING

In [ ]:
train_ds,test_ds = get_ds()

In [ ]:
!rm -r "/content/logs"

In [ ]:
tf.keras.backend.clear_session()
model = ViT(num_heads = configs.num_heads,
                d_model = configs.d_model ,
                hidden_layer_shape = configs.hidden_layer_shape1,
                num_layers = configs.num_layers,
                patch_size = configs.patch_size,
                num_classes = configs.num_classes,
                dropout_rate = configs.dropout_rate,
                training = configs.training)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: configs.learning_rate*tf.math.exp(-0.01*epoch))
lr_onPlateu = tf.keras.callbacks.ReduceLROnPlateau(patience=4)
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer = tf.keras.optimizers.Adam(learning_rate=configs.learning_rate),
                  metrics = ["accuracy"])
model.fit(train_ds,epochs = 500,steps_per_epoch=100,validation_data = test_ds,callbacks = [tensorboard_callback,lr_scheduler,lr_onPlateu],verbose = 1)

In [ ]:
%tensorboard --logdir /content/logs